In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [3]:
class movie_recommendation_vec:
    def __init__(self, **kargs):
        self.topn = kargs.get('topn',10)
        self.vec_type = kargs.get('vec_type','tfidf')
        self.df = kargs.get('data', pd.read_csv('./data/merged.csv'))
        self.a, self.b, self.c = kargs.get('a', 0.6), kargs.get('b', 0.2), kargs.get('c', 0.2)
        self.vote_thres = kargs.get('vote_thres', 100)
        self.verbose = kargs.get('verbose', 1)
        
        self.cvec = CountVectorizer(min_df=0, ngram_range=(1,2))
        self.stops = []
        with open('./data/total_stopwords', encoding='utf-8') as f:
            self.stops.append(f.readline()[:-2])
        
        if self.vec_type == 'tfidf':
            self.vec = TfidfVectorizer(analyzer='word', ngram_range=(1,5),stop_words=self.stops,
                       min_df=10, max_df=0.95, max_features=10000)
        elif self.vec_type == 'count':
            self.vec = CountVectorizer(analyzer='word',ngram_range=(1,1), stop_words=self.stops,
                      min_df=10, max_df=0.95, max_features=10000)
        else:
            raise ValueError('vec_type is not in [tfidf, count]')
            
        if self.verbose == 1:
            print('-'*35)
            print('# Parameters')
            print('      a, b, c        : {0}, {1}, {2}'.format(self.a, self.b, self.c))
            print('vote count threshold :', self.vote_thres)
            print('vec_type :',self.vec_type.capitalize())
            print('weighted_sum = vec_sim_scaled*{0}(a) + genre_scaled*{1}(b) + wvote_scaled*{2}(c)'.format(self.a, self.b, self.c))
            print('-'*35)
            
    def search_title(self, title_name):
        return self.df[self.df['title'].str.contains(title_name)].title
    
    def genre_sim_sorted(self, title_idx):
        genre_literal = self.df['genre'].apply(lambda x: x.replace('|',' '))
        genre = self.cvec.fit_transform(genre_literal)
        genre_sim = cosine_similarity(genre,genre)

        return np.array([(idx,sim) for idx,sim in enumerate(genre_sim[title_idx])])            

    def raw_to_vec(self, data_preprocess):
        return self.vec.fit_transform(data_preprocess)
    
    def cos_sim(self, data_preprocess, title_idx):
        cos_sims = []
        data_transformed = self.raw_to_vec(data_preprocess)
        
        for df_idx, src_idx in zip(self.df.index, range(data_transformed.shape[0])):
            if df_idx != title_idx:
                cos_sims.append((df_idx, cosine_similarity(data_transformed[title_idx].reshape(1,-1),
                                                          data_transformed[src_idx].reshape(1,-1))))
        return cos_sims
    
    def similar_vec_movies(self, title_idx):
        idx_sims = np.array(self.cos_sim(self.df['plot_preprocessed_kkma'], title_idx))
        sims_scaled = MinMaxScaler().fit_transform(idx_sims[:,1].reshape(-1,1))
        idx_sims[:,1] = sims_scaled.reshape(-1)

        idx_sims = np.array(sorted(idx_sims, key=lambda x: x[1], reverse=True))
        result_df = self.df.loc[idx_sims[:,0]]
        result_df['vec_sim'] = idx_sims[:,1]
        
        return result_df[result_df['vote_count'] > self.vote_thres]
    
    def result_by_weights(self, dataf):
        dataf['weighted_sum'] = dataf['vec_sim_scaled']*self.a + dataf['genre_scaled']*self.b + dataf['wvote_scaled']*self.c
        
        return dataf.sort_values('weighted_sum', ascending=False)
    
    def result_by_weights_negative(self, dataf):
        dataf['weighted_sum'] = dataf['vec_sim_scaled']*self.a + dataf['genre_scaled']*self.b + dataf['wvote_scaled']*self.c
        
        return dataf.sort_values('weighted_sum', ascending=True)
    
    def getMovies_negative(self, title):
        # no title result
        try:title_idx = self.df[self.df['title']== title].index.values[0]
        except:
            raise ValueError('There is no such title name in data. Search with "search_title" function')

        # get movies
        result = self.similar_vec_movies(title_idx)
        
        # IMDB's weighted_vote
        def weighted_vote_average(record):
            v, r = record['vote_count'], record['rating']
            return (v/(v+m))*r + (m/(m+v))*c
        c = result['rating'].mean()
        m = result['vote_count'].quantile(.6)
        result['weighted_vote'] = result.apply(weighted_vote_average,axis=1)
        
        # merge with genre
        genre_sim = self.genre_sim_sorted(title_idx)
        Result = result.rename(columns={'key_0': 'key_00'})
        result_with_genre = pd.merge(Result, pd.Series(genre_sim[:,1], name='genre_sim'), left_on=Result.index, right_on=genre_sim[:,0],)
        
        # minmax scale
        result_with_genre['vec_sim_scaled'] = MinMaxScaler().fit_transform(result_with_genre['vec_sim'].values.reshape(-1,1))
        result_with_genre['wvote_scaled'] = MinMaxScaler().fit_transform(result_with_genre['weighted_vote'].values.reshape(-1,1))
        result_with_genre['genre_scaled'] = MinMaxScaler().fit_transform(result_with_genre['genre_sim'].values.reshape(-1,1))

        # (optional)remove data genre score is 0
        no_genre_score_idx = result_with_genre[result_with_genre['genre_sim'] == 0].index
        result_with_genre.drop(no_genre_score_idx, inplace=True)
        
        result_with_genre = self.result_by_weights_negative(result_with_genre)
        return result_with_genre.head(self.topn)
    
    def getMovies(self, title):
        # no title result
        try:title_idx = self.df[self.df['title']== title].index.values[0]
        except:
            raise ValueError('There is no such title name in data. Search with "search_title" function')

        # get movies
        result = self.similar_vec_movies(title_idx)
        
        # IMDB's weighted_vote
        def weighted_vote_average(record):
            v, r = record['vote_count'], record['rating']
            return (v/(v+m))*r + (m/(m+v))*c
        c = result['rating'].mean()
        m = result['vote_count'].quantile(.6)
        result['weighted_vote'] = result.apply(weighted_vote_average,axis=1)
        
        # merge with genre
        genre_sim = self.genre_sim_sorted(title_idx)
        Result = result.rename(columns={'key_0': 'key_00'})
        result_with_genre = pd.merge(Result, pd.Series(genre_sim[:,1], name='genre_sim'), left_on=Result.index, right_on=genre_sim[:,0],)
        
        # minmax scale
        result_with_genre['vec_sim_scaled'] = MinMaxScaler().fit_transform(result_with_genre['vec_sim'].values.reshape(-1,1))
        result_with_genre['wvote_scaled'] = MinMaxScaler().fit_transform(result_with_genre['weighted_vote'].values.reshape(-1,1))
        result_with_genre['genre_scaled'] = MinMaxScaler().fit_transform(result_with_genre['genre_sim'].values.reshape(-1,1))

        # (optional)remove data genre score is 0
        no_genre_score_idx = result_with_genre[result_with_genre['genre_sim'] == 0].index
        result_with_genre.drop(no_genre_score_idx, inplace=True)
        
        result_with_genre = self.result_by_weights(result_with_genre)
        return result_with_genre.head(self.topn)

In [4]:
recom = movie_recommendation_vec(vec_type='tfidf')

-----------------------------------
# Parameters
      a, b, c        : 0.6, 0.2, 0.2
vote count threshold : 100
vec_type : Tfidf
weighted_sum = vec_sim_scaled*0.6(a) + genre_scaled*0.2(b) + wvote_scaled*0.2(c)
-----------------------------------


In [5]:
result = recom.getMovies(title='아이언맨 2')

In [6]:
result['title']

0                     아이언맨 3
1                       아이언맨
2                        로보캅
4                       어벤져스
3                        앤트맨
5                 스파이더맨: 홈커밍
203             스타 트렉: 더 비기닝
457     스타워즈 에피소드 3 - 시스의 복수
1598                    아일랜드
187            매트릭스 2 - 리로디드
Name: title, dtype: object

In [7]:
result = recom.getMovies_negative(title='아이언맨 2')

In [8]:
result

,key_0,key_00,Unnamed: 0,title,genre,year,date,rating,vote_count,plot,...,img_url,keywords,plot_preprocessed_kkma,vec_sim,weighted_vote,genre_sim,vec_sim_scaled,wvote_scaled,genre_scaled,weighted_sum
4334,6959,6959,6960,쾌걸 조로리의 대대대대모험,애니메이션|모험,2013,6.05,1.27,2599,"아주 오랜 옛날, 해적들이 수많은 보물을 숨겨 둔 전설의 가파르산! 가파르산의 작은...",...,https://movie-phinf.pstatic.net/20130513_201/1...,"['해적', '모아', '사이에서', '수수께끼', '전설', '마을', '힘', ...",옛날 해적 보물 숨기 전설 가 파 르 산 마을 가 파파 조용하 덜 마을 대소동 일어...,0.0246006,2.762994,0.258199,0.024601,0.051747,0.258199,0.076749
5050,10596,10596,10599,썬데이 서울,코미디|SF|판타지,2006,2.09,3.15,1865,생긴 것도 억울한데 왕따까지 당하는 반친구 도연(봉태규 분)에게 일어난 엄청난 신체...,...,https://movie-phinf.pstatic.net/20111223_167/1...,"['연', '배달', '이야기', '무술', '무협', '에피소드', '짜장면', ...",생기 왕따 당하 일어나 신체적 변화 짜장면 배달 가 사건 현장 살해 일어나 사건 고...,0.0186208,4.490493,0.200000,0.018621,0.298706,0.200000,0.110914
5783,9057,9057,9059,나는 비와 함께 간다,스릴러|범죄|액션,2009,10.15,3.77,2438,전직 형사 클라인(조쉬 하트넷)은 어느 날 대부호로부터 실종된 아들을 찾아달라는 의...,...,https://movie-phinf.pstatic.net/20111223_83/13...,"['실종', '타오', '마피아', '쿠', '대부호', '릴리', '홍콩', '남...",전직 날 대부 호로 실종 아들 찾 닿 의뢰 받 이름 시 타 오 기무 타 야 시 타 ...,0.0121572,4.720168,0.200000,0.012157,0.331540,0.200000,0.113602
4987,10110,10110,10113,상사부일체 - 두사부일체 3,코미디|액션,2007,9.19,3.25,2780,드디어 대학교 졸업장을 따고 강남을 맡게 된 계두식. 조직의 구조를 글로벌 하게 만...,...,https://movie-phinf.pstatic.net/20111223_112/1...,"['조직', '회사', '실', '조직원', '보험', '글로벌', '기대', '기...",대학교 졸업장 따 강남 맡 되 계두 식 조직 구조 글로벌 하 만들 큰형 하명 따르 ...,0.019061,4.225461,0.258199,0.019061,0.260817,0.258199,0.115240
4089,5871,5871,5872,조선미녀삼총사,코미디|액션|드라마,2014,1.29,4.04,4907,죄명 불문! 상대 불문! 완벽한 검거율을 자랑하는 조선 팔도 최고의 현상금 사냥꾼이...,...,https://movie-phinf.pstatic.net/20140108_218/1...,"['자랑', '조선', '주먹', '시크', '한', '리더', '실력파', '팔도...",죄명 불문 상대 불문 완벽 검거 자랑 조선 팔도 최고 현상금 사냥꾼 나타나 으뜸가 ...,0.0264315,4.541221,0.200000,0.026431,0.305958,0.200000,0.117050
6599,11573,11573,11577,포가튼,드라마|미스터리|SF|스릴러,2004,12.03,4.47,965,비행기 사고로 아들을 잃은 불행한 기억으로 괴로워 하던 텔리(줄리언 무어)는 정신과...,...,https://movie-phinf.pstatic.net/20111222_25/13...,"['아들', '기억', '정신', '남편', '샘', '상상', '속', '존재',...",비행기 사고 아들 잃 불행 기억 정신과 상담 치료 시작하 슬픔 지우 위하 행복 기억...,0.00134798,5.896527,0.169031,0.001348,0.499709,0.169031,0.134557
6376,10377,10377,10380,게드전기 - 어스시의 전설,애니메이션|판타지|가족|모험,2006,8.10,4.99,1705,용이 출몰하고 마법이 존재하는 ‘어스시(E rthse )’의 세계에서 펼쳐지는 마법...,...,https://movie-phinf.pstatic.net/20111221_227/1...,"['누', '바닷가', '테', '하', '출몰', '마법', '존재', '미국인'...",용 출몰 마법 존재 어스 시 의 세계 펼쳐지 마법사 왕자 모험 이야기 미국인 여성 ...,0.00512937,5.829023,0.169031,0.005129,0.490059,0.169031,0.134896
4328,48,48,48,국제수사,액션|드라마|범죄,2020,9.29,4.64,3312,필리핀으로 인생 첫 해외여행을 떠난 대천경찰서 강력팀 ‘홍병수’(곽도원) 경장. 여...,...,https://movie-phinf.pstatic.net/20200916_163/1...,"['수사', '범죄', '형사', '용의자', '의', '필리핀', '꿈', '마음...",필리핀 인생 해외여행 떠나 대천 경찰서 강력 팀 경장 여행 단꿈 잠시 범죄 조직 킬...,0.0246543,5.219072,0.200000,0.024654,0.402862,0.200000,0.135365
6606,7773,7773,7774,타이타닉 2,액션|모험,2012,4.25,2.23,2384,"타이타닉호 침몰이 있은 후 백 년 뒤, 예전의 타이타닉호 보다 한층 고급스럽고 거대...",...,https://movie-phinf.pstatic.net/20120412_116/1...,"['항해', '엔진', '안전', '대서양', '횡단', '축하', '진행', '기...",타이 타 호 침몰 있 후 뒤 예전 타이 타 호 고급 거대 최첨단 선박 타이 타 이름...,0.00103771,3.582860,0.516398,0.001038,0.168953,0.516398,0.137693
5081,4196,4196,4197,나의 절친 악당들,액션|범죄|드라마,2015,6.25,4.81,2793,인턴 지누에게 첫 번째 임무가 내려진다. 그동안 감시해온 차량의 이동라인을 완벽하게...,...,https://movie-phinf.pstatic.net/20150513_61/14...,"['차량', '파', '표적', '위험천만', '라인', '완벽', '충돌', '악...",일 턴 누 임무 내려지 그동안 감시 오 차량 이동 라인 완벽 파악 보 하 뒤 쫓 차...,0.0183793,5.434926,0.200000,0.018379,0.433720,0.200000,0.137772


In [21]:
result = recom.getMovies("어바웃 타임")
result['title']

0        베스트 오브 미
1         고양이 장례식
6       그 남자의 사랑법
3         사랑해, 파리
2            아일랜드
9       번지 점프를 하다
4     날 미치게 하는 남자
18          오 루시!
14          스타더스트
58           노팅 힐
Name: title, dtype: object

In [22]:
for i in result['title'] :
    print(i)

베스트 오브 미
고양이 장례식
그 남자의 사랑법
사랑해, 파리
아일랜드
번지 점프를 하다
날 미치게 하는 남자
오 루시!
스타더스트
노팅 힐


In [38]:
import csv
from tqdm.autonotebook import tqdm

movie = pd.read_csv('./data/merged.csv')
recom = movie_recommendation_vec(vec_type='tfidf')
columns = ['1st_movie', '2nd_movie', '3rd_movie', '4th_movie', '5th_movie', '6th_movie', '7th_movie', '8th_movie', '9th_movie', '10th_movie']
error = 0
for movie_idx in tqdm(range(movie['title'])) :
    try :
        results = recom.getMovies(movie['title'][movie_idx])
        data = {}
        index = 0
        for result in results['title'] :
            data["title"] = movie['title'][movie_idx]
            data[columns[index]] = result
            index += 1
        with open("./data/recommend_movie.csv", "a", encoding="utf-8", newline="") as csvfile :
            fieldnames = ['title'] + columns
            csvwriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
            csvwriter.writerow(data)
    except :
        error += 1
        print("error is "+ error)
        continue

-----------------------------------
# Parameters
      a, b, c        : 0.6, 0.2, 0.2
vote count threshold : 100
vec_type : Tfidf
weighted_sum = vec_sim_scaled*0.6(a) + genre_scaled*0.2(b) + wvote_scaled*0.2(c)
-----------------------------------


KeyboardInterrupt: 